In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_pl = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/t20_pl_g.csv')

/Users/roumyadas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
df_pl['phase'] = np.where(df_pl['ball']<7, 'pp', 
                        np.where(df_pl['ball']>=15, 'death',
                            'middle'))

df_pl['phase'].value_counts()

middle    156811
pp        125357
death      85015
Name: phase, dtype: int64

In [3]:
df = df_pl.sample(frac=1)

In [4]:
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,year,legality,bowler_wicket,islegal,isDotforBatter,isOne,isTwo,isThree,isStrikeRotation,isFour,isSix,isBowlerWicket,is_striker_Out,is_nonstriker_Out,isWicket,total_runs,runs_scored,wickets_down,legal_balls_bowled,lbb_fl,run_rate,target,legal_balls_remaining,runs_remaining,reqd_run_rate,last_fow_run_rate,last_fow,partnership_runs,striker_runs,non_striker_runs,isno,is_faced_by_batter,striker_balls,non_striker_balls,fours,sixes,striker_batting_position,non_striker_batting_position,phase
115294,1359486,2023,2023-04-08,MI,1,0.5,Mumbai Indians,Chennai Super Kings,Ishan Kishan,RG Sharma,DL Chahar,1,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2023,True,False,1,0,1,0,0,1,0,0,0,0,0,0,1,6,0,5,True,7.200000,NaN,115,NaN,NaN,NaN,0.0,163.0,1,5,False,True,1,4,1,0,2,1,pp
245315,1280246,2021/22,2021-11-06,DC,2,11.5,Assam,Bihar,Amlanjyoti Das,R Parag,Malay Raj,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2021,True,False,1,1,0,0,0,0,0,0,0,0,0,0,0,87,6,71,True,7.352113,136.0,49,49.0,6.000000,7.238095,76.0,109.0,4,32,False,True,6,18,9,4,8,4,middle
280903,1332938,2022/23,2022-10-12,RR,2,16.2,Puducherry,Hyderabad (India),Mohit Mittan,Parameeswaran Sivaraman,Ravi Teja,1,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2022,True,False,1,0,1,0,0,1,0,0,0,0,0,0,1,108,3,98,True,6.612245,148.0,22,40.0,10.909091,6.700000,67.0,4.0,26,16,False,True,17,23,7,4,5,4,death
62725,1289610,2021/22,2022-01-20,"Eden Park Outer Oval, Auckland",1,10.3,Auckland,Wellington,GD Phillips,BJ Horne,BV Sears,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2022,True,False,1,1,0,0,0,0,0,0,0,0,0,0,0,48,4,63,True,4.571429,NaN,57,NaN,NaN,4.867925,43.0,113.0,9,3,False,True,14,5,3,0,4,6,middle
284838,1341195,2022/23,2022-12-24,"Basin Reserve, Wellington",2,8.5,Otago,Wellington,DN Phillips,D Foxcroft,R Ravindra,1,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2022,True,False,1,0,1,0,0,1,0,0,0,0,0,0,1,57,1,53,True,6.452830,153.0,67,96.0,8.597015,3.000000,57.0,130.0,21,28,False,True,27,22,5,0,2,3,middle


In [5]:
df_1 = df[df.innings==1]
df_1.shape

(192539, 61)

In [6]:
df_2 = df[df.innings==2]
df_2.shape

(174644, 61)

In [8]:
player_list = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/player_list.xlsx',
                           sheet_name='season_03')

player_list.iloc[130:135]

players_list = player_list['Func_Name'].dropna().tolist()

#players_list = players_list.remove('nan')

print(type(players_list))

len(players_list)

<class 'list'>


247

In [47]:
#### innings 1 stuff, no run rate required

In [48]:
df_1_b = df_1[df_1.striker.isin(players_list)]

df_1_b.shape

(117458, 61)

In [49]:
batter_stats = df_1_b.groupby(['striker','phase','innings']).agg(
    num_innings = ('match_id','nunique'),
    runs = ('total_runs','sum'),
    balls = ('is_faced_by_batter' ,'sum'),
    outs = ('is_striker_Out','sum'),
    fours = ('isFour', 'sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum'),
    
    ones = ('isOne','sum'),
    twos = ('isTwo','sum'),
    threes = ('isThree','sum')
    
    
    
    
).reset_index()

batter_stats['strike_rate'] = 100*batter_stats['runs']/batter_stats['balls']
batter_stats['balls_per_dismissal'] = batter_stats['balls']/batter_stats['outs']
batter_stats['bpb'] = batter_stats['balls']/(batter_stats['fours']+batter_stats['sixes'])
batter_stats['dot_%'] = 100*batter_stats['dots']/batter_stats['balls']

batter_stats['out_pb'] = 1/batter_stats['balls_per_dismissal']

batter_stats['one_pb'] = batter_stats['ones']/batter_stats['balls']
batter_stats['two_pb'] = batter_stats['twos']/batter_stats['balls']
batter_stats['three_pb'] = batter_stats['threes']/batter_stats['balls']
batter_stats['four_pb'] = batter_stats['fours']/batter_stats['balls']
batter_stats['six_pb'] = batter_stats['sixes']/batter_stats['balls']
batter_stats['dot_pb'] = batter_stats['dots']/batter_stats['balls']


pb_cols = [column for column in batter_stats.columns if '_pb' in str(column)]
pb_cols_div = [column for column in batter_stats.columns if '_pb' in str(column) and column!='out_pb']
print(len(pb_cols))


7


In [50]:
batter_stats['pb_sum'] = batter_stats[pb_cols].apply(lambda x: x.sum(), axis=1)

batter_stats['out_prob'] = batter_stats['out_pb']/1

for col in pb_cols_div:
    rem_sum = (batter_stats['pb_sum']-batter_stats['out_pb'])
    multiplier = (1-batter_stats['out_prob'])/rem_sum
    batter_stats[str(col).replace('_pb','_prob')] = batter_stats[col]*multiplier

In [51]:
batter_stats.tail(50)

,striker,phase,innings,num_innings,runs,balls,outs,fours,sixes,dots,ones,twos,threes,strike_rate,balls_per_dismissal,bpb,dot_%,out_pb,one_pb,two_pb,three_pb,four_pb,six_pb,dot_pb,pb_sum,out_prob,one_prob,two_prob,three_prob,four_prob,six_prob,dot_prob
461,TH David,death,1,78,1628,780,50,95,119,284,249,92,2,208.717949,15.600000,3.644860,36.410256,0.064103,0.319231,0.117949,0.002564,0.121795,0.152564,0.364103,1.142308,0.064103,0.277097,0.102381,0.002226,0.105720,0.132428,0.316046
462,TH David,middle,1,71,996,696,18,67,49,230,328,37,2,143.103448,38.666667,6.000000,33.045977,0.025862,0.471264,0.053161,0.002874,0.096264,0.070402,0.330460,1.050287,0.025862,0.448131,0.050551,0.002733,0.091539,0.066946,0.314238
463,TH David,pp,1,16,165,126,6,24,1,55,46,4,0,130.952381,21.000000,5.040000,43.650794,0.047619,0.365079,0.031746,0.000000,0.190476,0.007937,0.436508,1.079365,0.047619,0.336996,0.029304,0.000000,0.175824,0.007326,0.402930
464,TM Head,death,1,8,129,89,5,5,5,24,42,14,1,144.943820,17.800000,8.900000,26.966292,0.056180,0.471910,0.157303,0.011236,0.056180,0.056180,0.269663,1.078652,0.056180,0.435609,0.145203,0.010372,0.051858,0.051858,0.248920
465,TM Head,middle,1,24,477,315,17,35,21,120,132,25,1,151.428571,18.529412,5.625000,38.095238,0.053968,0.419048,0.079365,0.003175,0.111111,0.066667,0.380952,1.114286,0.053968,0.373881,0.070811,0.002832,0.099135,0.059481,0.339892
466,TM Head,pp,1,35,778,464,17,94,33,186,132,23,2,167.672414,27.294118,3.653543,40.086207,0.036638,0.284483,0.049569,0.004310,0.202586,0.071121,0.400862,1.049569,0.036638,0.270561,0.047143,0.004099,0.192672,0.067640,0.381245
467,TU Deshpande,death,1,5,6,10,3,0,0,9,1,0,1,60.000000,3.333333,inf,90.000000,0.300000,0.100000,0.000000,0.100000,0.000000,0.000000,0.900000,1.400000,0.300000,0.063636,0.000000,0.063636,0.000000,0.000000,0.572727
468,Tanush Kotian,death,1,2,27,16,1,3,1,7,4,2,0,168.750000,16.000000,4.000000,43.750000,0.062500,0.250000,0.125000,0.000000,0.187500,0.062500,0.437500,1.125000,0.062500,0.220588,0.110294,0.000000,0.165441,0.055147,0.386029
469,Tanush Kotian,middle,1,1,10,14,0,0,0,7,5,2,0,71.428571,inf,inf,50.000000,0.000000,0.357143,0.142857,0.000000,0.000000,0.000000,0.500000,1.000000,0.000000,0.357143,0.142857,0.000000,0.000000,0.000000,0.500000
470,Tilak Varma,death,1,16,344,159,8,23,21,58,59,17,1,216.352201,19.875000,3.613636,36.477987,0.050314,0.371069,0.106918,0.006289,0.144654,0.132075,0.364780,1.176101,0.050314,0.313025,0.090194,0.005306,0.122027,0.111416,0.307719


In [52]:
prob_cols = [column for column in batter_stats.columns if '_prob' in str(column)]

for col in batter_stats.columns[[0,1,2,3,4,5,6,13]]:
    prob_cols.append(col)

batter_stats_trimmed = batter_stats[prob_cols]

In [53]:
batter_stats_trimmed[batter_stats_trimmed.striker=='V Kohli']

,out_prob,one_prob,two_prob,three_prob,four_prob,six_prob,dot_prob,striker,phase,innings,num_innings,runs,balls,outs,strike_rate
479,0.049383,0.350929,0.146591,0.000000,0.124380,0.104390,0.224328,V Kohli,death,1,35,828,405,20,204.444444
480,0.027607,0.525906,0.083293,0.001937,0.062954,0.030993,0.267311,V Kohli,middle,1,61,1219,978,27,124.642127
481,0.033994,0.375767,0.053175,0.002659,0.116098,0.024815,0.393492,V Kohli,pp,1,83,1308,1059,36,123.512748


In [54]:
batter_stats_trimmed.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_inn1.csv',
                           index=None)

In [55]:
#### innings 2 stuff, bucketing based on run rate required, balls remain, wickets lost

In [56]:
df_2_b = df_2[df_2.striker.isin(players_list)]

df_2_b.shape

(104957, 61)

In [57]:
df_2_b['nrr_phase'] = np.where(df_2_b['reqd_run_rate']>=10, 'crucial', 
                        np.where(df_2_b['reqd_run_rate']<=8, 'easy',
                            'moderate'))

df_2_b['wkt_phase'] = np.where(df_2_b['wickets_down']>=7, 'tough', 
                        np.where(df_2_b['wickets_down']<=3, 'easy',
                            'medium'))

df_2_b['nrr_phase'].value_counts(), df_2_b['wkt_phase'].value_counts()

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_78233/366456033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_b['nrr_phase'] = np.where(df_2_b['reqd_run_rate']>=10, 'crucial',
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_78233/366456033.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_b['wkt_phase'] = np.where(df_2_b['wickets_down']>=7, 'tough',


(easy        40239
 crucial     38199
 moderate    26519
 Name: nrr_phase, dtype: int64,
 easy      75340
 medium    23669
 tough      5948
 Name: wkt_phase, dtype: int64)

In [58]:
df_2_b.columns
#wickets_down, reqd_run_rate

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'year', 'legality', 'bowler_wicket',
       'islegal', 'isDotforBatter', 'isOne', 'isTwo', 'isThree',
       'isStrikeRotation', 'isFour', 'isSix', 'isBowlerWicket',
       'is_striker_Out', 'is_nonstriker_Out', 'isWicket', 'total_runs',
       'runs_scored', 'wickets_down', 'legal_balls_bowled', 'lbb_fl',
       'run_rate', 'target', 'legal_balls_remaining', 'runs_remaining',
       'reqd_run_rate', 'last_fow_run_rate', 'last_fow', 'partnership_runs',
       'striker_runs', 'non_striker_runs', 'isno', 'is_faced_by_batter',
       'striker_balls', 'non_striker_balls', 'fours', 'sixes',
       'striker_batting_position', 'non_striker_batting_position', 'phase',
      

In [59]:
batter_stats = df_2_b.groupby(['striker','innings','phase','nrr_phase','wkt_phase']).agg(
    num_innings = ('match_id','nunique'),
    runs = ('total_runs','sum'),
    balls = ('is_faced_by_batter' ,'sum'),
    outs = ('is_striker_Out','sum'),
    fours = ('isFour', 'sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum'),
    
    ones = ('isOne','sum'),
    twos = ('isTwo','sum'),
    threes = ('isThree','sum')
    
    
    
    
).reset_index()

batter_stats['strike_rate'] = 100*batter_stats['runs']/batter_stats['balls']
batter_stats['balls_per_dismissal'] = batter_stats['balls']/batter_stats['outs']
batter_stats['bpb'] = batter_stats['balls']/(batter_stats['fours']+batter_stats['sixes'])
batter_stats['dot_%'] = 100*batter_stats['dots']/batter_stats['balls']

batter_stats['out_pb'] = 1/batter_stats['balls_per_dismissal']

batter_stats['one_pb'] = batter_stats['ones']/batter_stats['balls']
batter_stats['two_pb'] = batter_stats['twos']/batter_stats['balls']
batter_stats['three_pb'] = batter_stats['threes']/batter_stats['balls']
batter_stats['four_pb'] = batter_stats['fours']/batter_stats['balls']
batter_stats['six_pb'] = batter_stats['sixes']/batter_stats['balls']
batter_stats['dot_pb'] = batter_stats['dots']/batter_stats['balls']


pb_cols = [column for column in batter_stats.columns if '_pb' in str(column)]
pb_cols_div = [column for column in batter_stats.columns if '_pb' in str(column) and column!='out_pb']
print(len(pb_cols))


7


In [60]:
batter_stats['pb_sum'] = batter_stats[pb_cols].apply(lambda x: x.sum(), axis=1)

batter_stats['out_prob'] = batter_stats['out_pb']/1

for col in pb_cols_div:
    rem_sum = (batter_stats['pb_sum']-batter_stats['out_pb'])
    multiplier = (1-batter_stats['out_prob'])/rem_sum
    batter_stats[str(col).replace('_pb','_prob')] = batter_stats[col]*multiplier

In [61]:
batter_stats.tail(50)

,striker,innings,phase,nrr_phase,wkt_phase,num_innings,runs,balls,outs,fours,sixes,dots,ones,twos,threes,strike_rate,balls_per_dismissal,bpb,dot_%,out_pb,one_pb,two_pb,three_pb,four_pb,six_pb,dot_pb,pb_sum,out_prob,one_prob,two_prob,three_prob,four_prob,six_prob,dot_prob
2211,WG Jacks,2,pp,moderate,easy,40,605,391,19,66,25,178,110,23,1,154.731458,20.578947,4.296703,45.524297,0.048593,0.281330,0.058824,0.002558,0.168798,0.063939,0.455243,1.079284,0.048593,0.259689,0.054299,0.002361,0.155813,0.059020,0.420224
2212,WG Jacks,2,pp,moderate,medium,1,1,2,0,0,0,1,1,0,0,50.000000,inf,inf,50.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.500000,1.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.500000
2213,Washington Sundar,2,death,crucial,medium,10,134,70,4,13,5,19,28,8,1,191.428571,17.500000,3.888889,27.142857,0.057143,0.400000,0.114286,0.014286,0.185714,0.071429,0.271429,1.114286,0.057143,0.356757,0.101931,0.012741,0.165637,0.063707,0.242085
2214,Washington Sundar,2,death,crucial,tough,8,66,40,8,5,5,28,4,2,0,165.000000,5.000000,4.000000,70.000000,0.200000,0.100000,0.050000,0.000000,0.125000,0.125000,0.700000,1.300000,0.200000,0.072727,0.036364,0.000000,0.090909,0.090909,0.509091
2215,Washington Sundar,2,death,easy,easy,1,15,7,0,1,1,0,5,0,0,214.285714,inf,3.500000,0.000000,0.000000,0.714286,0.000000,0.000000,0.142857,0.142857,0.000000,1.000000,0.000000,0.714286,0.000000,0.000000,0.142857,0.142857,0.000000
2216,Washington Sundar,2,death,easy,medium,5,44,28,1,4,1,8,11,5,0,157.142857,28.000000,5.600000,28.571429,0.035714,0.392857,0.178571,0.000000,0.142857,0.035714,0.285714,1.071429,0.035714,0.365764,0.166256,0.000000,0.133005,0.033251,0.266010
2217,Washington Sundar,2,death,moderate,tough,1,10,8,0,1,0,7,3,0,0,125.000000,inf,8.000000,87.500000,0.000000,0.375000,0.000000,0.000000,0.125000,0.000000,0.875000,1.375000,0.000000,0.272727,0.000000,0.000000,0.090909,0.000000,0.636364
2218,Washington Sundar,2,middle,crucial,easy,3,11,17,1,0,0,6,11,0,0,64.705882,17.000000,inf,35.294118,0.058824,0.647059,0.000000,0.000000,0.000000,0.000000,0.352941,1.058824,0.058824,0.608997,0.000000,0.000000,0.000000,0.000000,0.332180
2219,Washington Sundar,2,middle,crucial,medium,12,72,74,5,5,0,30,37,4,1,97.297297,14.800000,14.800000,40.540541,0.067568,0.500000,0.054054,0.013514,0.067568,0.000000,0.405405,1.108108,0.067568,0.448052,0.048438,0.012110,0.060548,0.000000,0.363285
2220,Washington Sundar,2,middle,crucial,tough,1,6,4,0,1,0,1,2,0,0,150.000000,inf,4.000000,25.000000,0.000000,0.500000,0.000000,0.000000,0.250000,0.000000,0.250000,1.000000,0.000000,0.500000,0.000000,0.000000,0.250000,0.000000,0.250000


In [62]:
prob_cols = [column for column in batter_stats.columns if '_prob' in str(column)]

for col in batter_stats.columns[[0,1,2,3,4,5,6,7,8,15]]:
    prob_cols.append(col)

batter_stats_trimmed_2 = batter_stats[prob_cols]

In [63]:
batter_stats_trimmed_2[batter_stats_trimmed_2.striker=='V Kohli']

,out_prob,one_prob,two_prob,three_prob,four_prob,six_prob,dot_prob,striker,innings,phase,nrr_phase,wkt_phase,num_innings,runs,balls,outs,strike_rate
2126,0.083333,0.352564,0.211538,0.000000,0.070513,0.000000,0.282051,V Kohli,2,death,crucial,easy,4,17,12,1,141.666667
2127,0.071429,0.282609,0.121118,0.000000,0.161491,0.121118,0.242236,V Kohli,2,death,crucial,medium,6,99,42,3,235.714286
2128,0.000000,0.363636,0.090909,0.000000,0.090909,0.181818,0.272727,V Kohli,2,death,crucial,tough,1,23,11,0,209.090909
2129,0.040000,0.432941,0.056471,0.018824,0.103529,0.112941,0.235294,V Kohli,2,death,easy,easy,14,183,100,4,183.000000
2130,0.076923,0.071006,0.213018,0.000000,0.142012,0.142012,0.355030,V Kohli,2,death,easy,medium,4,31,13,1,238.461538
2131,0.000000,0.500000,0.000000,0.000000,0.000000,0.055556,0.444444,V Kohli,2,death,moderate,easy,7,17,16,0,106.250000
2132,0.000000,0.500000,0.125000,0.000000,0.000000,0.250000,0.125000,V Kohli,2,death,moderate,medium,1,18,8,0,225.000000
2133,0.023529,0.548579,0.093258,0.000000,0.065829,0.043886,0.224917,V Kohli,2,middle,crucial,easy,13,242,170,4,142.352941
2134,0.033898,0.490970,0.079189,0.000000,0.047513,0.063351,0.285079,V Kohli,2,middle,crucial,medium,7,80,59,2,135.593220
2135,0.013333,0.499853,0.099971,0.008693,0.091278,0.043465,0.243407,V Kohli,2,middle,easy,easy,21,315,225,3,140.000000


In [64]:
batter_stats_trimmed_2.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_inn2.csv',
                           index=None)

batter_stats_trimmed_2[batter_stats_trimmed_2.out_prob>1]